In [2]:
#importing numy, pandas, count Vectorizer,GridSearchCV,
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV,KFold

In [3]:
# getting traning and testing datasets in pandas 
training_data = pd.read_csv("twitter_train_data.csv",delimiter=',')
testing_data = pd.read_csv("twitter_test_data.csv",delimiter=',')

In [4]:
#viewing training data by using describe function
training_data.describe(include='all')

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
count,1.098000e+04,10980,10980,31,10980,24,10980.000000,10980,776,10980,7430,7403
unique,NaN,3,6,3,6438,11,NaN,10851,632,10758,2658,78
top,NaN,negative,United,negative,JetBlueNews,Customer Service Issue,NaN,@united thanks,"[0.0, 0.0]",2015-02-24 11:38:11 -0800,"New York, NY",Eastern Time (US & Canada)
freq,NaN,6851,2928,24,43,9,NaN,6,131,3,125,2819
mean,5.692169e+17,NaN,NaN,NaN,NaN,NaN,0.080965,NaN,NaN,NaN,NaN,NaN
std,7.795438e+14,NaN,NaN,NaN,NaN,NaN,0.740303,NaN,NaN,NaN,NaN,NaN
min,5.675883e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,5.685584e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
50%,5.694753e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
75%,5.698902e+17,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [5]:
#viewing testing data by using describe function
testing_data.describe(include='all')

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
count,3.660000e+03,3660,9,3660,8,3660.000000,3660,243,3660,2477,2417
unique,NaN,6,2,2805,6,NaN,3650,209,3635,1258,59
top,NaN,United,negative,JetBlueNews,Customer Service Issue,NaN,@AmericanAir Aww cool! It's nice to know they ...,"[0.0, 0.0]",2015-02-24 09:54:34 -0800,USA,Eastern Time (US & Canada)
freq,NaN,894,8,20,3,NaN,2,33,3,43,925
mean,5.692226e+17,NaN,NaN,NaN,NaN,0.087705,NaN,NaN,NaN,NaN,NaN
std,7.779030e+14,NaN,NaN,NaN,NaN,0.762048,NaN,NaN,NaN,NaN,NaN
min,5.675924e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
25%,5.685633e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
50%,5.694842e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
75%,5.698927e+17,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [9]:
xtrain=training_data["text"]

In [10]:
training_data["airline_sentiment"]

0        negative
1        positive
2        positive
3        negative
4        negative
5        negative
6        negative
7        positive
8        negative
9        positive
10       negative
11        neutral
12       positive
13       negative
14       negative
15       negative
16       negative
17       negative
18       negative
19       negative
20       negative
21        neutral
22       negative
23       negative
24       positive
25        neutral
26       negative
27       negative
28       positive
29       positive
           ...   
10950    positive
10951    negative
10952    negative
10953    negative
10954    negative
10955    negative
10956     neutral
10957    negative
10958    negative
10959    negative
10960    negative
10961    negative
10962     neutral
10963    negative
10964    negative
10965     neutral
10966    negative
10967    negative
10968    negative
10969    positive
10970    negative
10971     neutral
10972    negative
10973    negative
10974    p

In [11]:
xtrain[0]

'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

CLEANING THE TRAINING DATA

In [12]:
# getting stopwords in stop as a set
from nltk.corpus import stopwords
import string
stop = set(stopwords.words("english"))
punctn = set(string.punctuation)
stop.update(punctn)

In [13]:
#Importing lemmatizer and pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag

In [14]:
#to get simple part of speech used by wordnet
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif(tag.startswith('V')):
        return wordnet.VERB
    elif(tag.startswith('N')):
        return wordnet.NOUN
    elif(tag.startswith('R')):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [15]:
#Lemmatization
twitter_texts=[]
lemmatizer = WordNetLemmatizer()
for texts in xtrain:
    words = word_tokenize(texts)
    lemmatized_texts = ""
    for word in words:
        pos = pos_tag([word])
        lemmatized_texts = lemmatized_texts + " " +lemmatizer.lemmatize(word,get_simple_pos(pos[0][1]))
    twitter_texts.append(lemmatized_texts)
        
        

In [14]:
# making an objet of count Vectorizer,passing the argument max_feature, ngram_range, stop_words, max_df,min_df,then fit transform 
#training data
count_vec = CountVectorizer(max_features=500,ngram_range=(1,3),stop_words=stop,max_df=0.90,min_df=0.001)
xtrain = count_vec.fit_transform(twitter_texts)
xtrain

<10980x500 sparse matrix of type '<class 'numpy.int64'>'
	with 80042 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer(max_features=500,ngram_range=(1,3),stop_words=stop,max_df=0.90,min_df=0.005)
xtrain2 = tfidf_vec.fit_transform(twitter_texts)
xtrain2

<10980x391 sparse matrix of type '<class 'numpy.float64'>'
	with 74909 stored elements in Compressed Sparse Row format>

In [73]:
# Getting test data in String of array because count vectorizer takes only in these formats after tranform it will give a sparse matrix 
twitter_texts_test=[]
for text in testing_data["text"]:
    twitter_texts_test.append(text)
xtest = count_vec.transform(twitter_texts_test)
xtest

<3660x500 sparse matrix of type '<class 'numpy.int64'>'
	with 22543 stored elements in Compressed Sparse Row format>

In [22]:
#Using TF IDF Vectorizer transforming test data
twitter_texts_test_tf=[]
for text in testing_data["text"]:
    twitter_texts_test_tf.append(text)
xtest2 = tfidf_vec.transform(twitter_texts_test_tf)
xtest2

<3660x391 sparse matrix of type '<class 'numpy.float64'>'
	with 21107 stored elements in Compressed Sparse Row format>

In [74]:
# GEtting list of feature name obtained from count vectorizer
count_vec.get_feature_names()

['10',
 '11',
 '12',
 '15',
 '1st',
 '20',
 '200',
 '24',
 '25',
 '30',
 '40',
 '45',
 '50',
 '800',
 'aa',
 'able',
 'account',
 'actually',
 'add',
 'agent',
 'ago',
 'air',
 'airline',
 'airport',
 'airways',
 'allow',
 'almost',
 'already',
 'also',
 'always',
 'amaze',
 'american',
 'americanair',
 'americanair flight',
 'americanair thank',
 'americanair thanks',
 'amp',
 'another',
 'answer',
 'anyone',
 'anything',
 'app',
 'appreciate',
 'around',
 'arrive',
 'ask',
 'assistance',
 'attendant',
 'available',
 'away',
 'awesome',
 'back',
 'bad',
 'bag',
 'baggage',
 'bc',
 'believe',
 'best',
 'big',
 'board',
 'boarding',
 'book',
 'book flight',
 'booking',
 'booking problems',
 'bos',
 'boston',
 'bring',
 'broken',
 'business',
 'ca',
 'ca get',
 'call',
 'call back',
 'cancelled',
 'cancelled flight',
 'cancelled flight flight',
 'cancelled flighted',
 'cancelled flighted flight',
 'cancelled flightled',
 'cancelled flightled flight',
 'car',
 'card',
 'care',
 'carry',
 

In [23]:
# GEtting list of feature name obtained from TFIDF vectorizer
tfidf_vec.get_feature_names()

['10',
 '11',
 '15',
 '1st',
 '20',
 '30',
 '40',
 '45',
 '50',
 'aa',
 'able',
 'actually',
 'add',
 'agent',
 'ago',
 'air',
 'airline',
 'airport',
 'allow',
 'almost',
 'already',
 'also',
 'always',
 'amaze',
 'american',
 'americanair',
 'americanair flight',
 'americanair thanks',
 'amp',
 'another',
 'answer',
 'anyone',
 'anything',
 'app',
 'appreciate',
 'arrive',
 'ask',
 'attendant',
 'available',
 'away',
 'awesome',
 'back',
 'bad',
 'bag',
 'baggage',
 'best',
 'big',
 'board',
 'boarding',
 'book',
 'book flight',
 'booking',
 'booking problems',
 'bos',
 'boston',
 'business',
 'ca',
 'ca get',
 'call',
 'call back',
 'cancelled',
 'cancelled flight',
 'cancelled flighted',
 'cancelled flightled',
 'cancelled flightled flight',
 'card',
 'care',
 'cause',
 'chance',
 'change',
 'change flight',
 'charge',
 'check',
 'checked',
 'chicago',
 'claim',
 'class',
 'clt',
 'co',
 'come',
 'company',
 'connect',
 'connect flight',
 'connection',
 'contact',
 'cost',
 'could'

GRIDSEARCHING ON DIFFERENT ALGO AND CHECKING WHICH GIVE A BETTER ACCURACY

1.SVC

In [24]:
#SVC
from sklearn.svm import SVC


In [ ]:
#gridsearch for SVC
clf_SVC = SVC()
grid_SVC = {'C':[10,1e2,5e2,1e3,5e3],'gamma':[1e-2,1e-3,1e-4,5e-3]}
gridsearch_SVC = GridSearchCV(clf_SVC,grid_SVC,cv=KFold(n_splits=3,shuffle=True,random_state=0))
gridsearch_SVC.fit(xtrain,ytrain)

In [ ]:
gridsearch_SVC.best_estimator_

In [ ]:
gridsearch_SVC.best_score_

In [ ]:
clf_SVC = SVC(C=10,gamma=0.01)
clf_SVC.fit(xtrain,ytrain)

BY USING TFIDF VECTORIZER

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf_SVC = SVC()
grid_SVC = {'C':[10,1e2,5e2,1e3,5e3],'gamma':[1e-2,1e-3,1e-4,5e-3]}
gridsearch_SVC = GridSearchCV(clf_SVC,grid_SVC,cv=KFold(n_splits=3,shuffle=True,random_state=0))
gridsearch_SVC.fit(xtrain2,ytrain)

In [ ]:
gridsearch_SVC.best_estimator_

In [ ]:
gridsearch_SVC.best_score_

In [16]:
ypred_SVC = clf_SVC.predict(xtest)


NameError: name 'clf_SVC' is not defined

In [192]:
np.savetxt("ypred_svc.csv",ypred_SVC,fmt='%s',delimiter=',')

2.KNN CLASSIFIER

In [176]:
from sklearn.neighbors import KNeighborsClassifier

In [177]:
clf_KNN = KNeighborsClassifier()
grid_KNN = {'n_neighbors':[11,13,15,17]}
gridsearch_KNN = GridSearchCV(clf_KNN,grid_KNN,cv=KFold(n_splits=3,shuffle=True,random_state=0))
gridsearch_KNN.fit(xtrain,ytrain)

GridSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [11, 13, 15, 17]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [178]:
gridsearch_KNN.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')

In [179]:
gridsearch_KNN.best_score_

0.58205828779599267

3.RANDOM FOREST CLASSIFIER

In [180]:
from sklearn.ensemble import RandomForestClassifier


In [181]:
clf_RFC = RandomForestClassifier(criterion="entropy")
grid_RFC = {"n_estimators":[5,10,15,20,25,30],"max_depth":[5,10,15,20,25,30,35]}
gridsearch_RFC=GridSearchCV(clf_RFC,grid_RFC,cv=KFold(n_splits=3,shuffle=True,random_state=0))
gridsearch_RFC.fit(xtrain,ytrain)

GridSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 15, 20, 25, 30], 'max_depth': [5, 10, 15, 20, 25, 30, 35]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [182]:
gridsearch_RFC.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=35, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [183]:
gridsearch_RFC.best_score_

0.72222222222222221